<a href="https://colab.research.google.com/github/Jaehyeonee/ktcloud_genai/blob/main/109_LLM_HuggingFace%EB%AA%A8%EB%8D%B8%ED%99%95%EC%9D%B8_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Hugging Face 모델 확인하기**

---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

## **1.Hugging Face 사용**

- **Hugging Face**
    - Hugging Face는 AI 모델과 데이터셋을 공유·활용할 수 있는 대표적인 오픈소스 플랫폼, 커뮤니티
        - **모델 허브(Model Hub)** : 수만 개의 공개된 머신러닝/딥러닝 모델을 누구나 다운로드·활용 가능
        - **데이터셋 허브(Datasets)** : 다양한 표준/비표준 데이터셋을 간편하게 불러와 실험 가능
        - **Transformers 라이브러리** : BERT, GPT, T5, LLaMA 등 최신 NLP·멀티모달 모델을 손쉽게 사용 가능
        - **PEFT/Accelerate 등 도구** : 파인튜닝·분산 학습·최적화 지원
        - **커뮤니티**: 연구자·개발자가 모델과 코드를 공유하고 협업할 수 있는 생태계

- **transformers**
    - transformers library https://huggingface.co/docs/transformers/index
    - Hugging Face에서 제공하는 사전 훈련된 AI 모델들을 쉽게 사용할 수 있게 해주는 파이썬 라이브러리
    - 복잡한 AI 모델을 간단한 코드로 사용할 수 있게 해주는 도구

- **pipeline**
    - https://huggingface.co/docs/transformers/main/en/main_classes/pipelines
    - pipeline은 자연어 처리(NLP)·비전·오디오 등 다양한 AI 태스크를 쉽게 실행할 수 있도록 미리 구성된 추론(실행) 도구
    - 태스크 이름(예: "sentiment-analysis", "translation")을 입력하면,자동으로 적합한 모델 + 토크나이저 + 전/후처리 로직을 불러와,사용자가 텍스트/이미지/오디오를 바로 넣어 결과를 얻을 수 있게 함

    - **pipeline(sentiment-analysis)**
        - Hugging Face Pipelines Documentation :  
        - 텍스트의 감정(긍정/부정) 분석에 특화된 즉시 사용 가능한 AI 모델을 사용하는 파이프라인의 작업(task) 이름
        - 감정 분석에 특화된 즉시 사용 가능한 AI 모델을 사용하는 작업(task) 이름
        - (AI모델 X) 내부적으로 BERT, RoBERTa, DistilBERT 등의 실제 신경망 모델이 동작


### **예제 1: 모델 정보 확인하기**

In [6]:
from huggingface_hub import model_info

# 예: BERT base uncased 모델 정보 불러오기
# info = model_info("bert-base-uncased")
info = model_info("google-bert/bert-base-uncased")
# info = model_info("Qwen/Qwen3-Omni-30B-A3B-Instruct")

# 전체 메타데이터 보기
print(info.cardData)

# 특정 항목 추출 (있을 경우만)
if "model_size" in info.cardData:
    print("모델 파라미터 수:", info.cardData["model_size"])
if "datasets" in info.cardData:
    print("학습 데이터셋:", info.cardData["datasets"])


datasets:
- bookcorpus
- wikipedia
language: en
license: apache-2.0
tags:
- exbert
학습 데이터셋: ['bookcorpus', 'wikipedia']


In [7]:
info.author

'google-bert'

In [8]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

# 모델과 토크나이저 로드
model_name = "google-bert/bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)

# 모델 구성 정보 출력
print("=== BERT Base Uncased 모델 정보 ===")
print(f"모델 이름: {model_name}")
print(f"어휘 사전 크기: {config.vocab_size}")
print(f"히든 레이어 수: {config.num_hidden_layers}")
print(f"히든 사이즈: {config.hidden_size}")
print(f"어텐션 헤드 수: {config.num_attention_heads}")
print(f"최대 시퀀스 길이: {config.max_position_embeddings}")

# 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
print(f"총 파라미터 수: {total_params:,} ({total_params/1e6:.1f}M)")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

=== BERT Base Uncased 모델 정보 ===
모델 이름: google-bert/bert-base-uncased
어휘 사전 크기: 30522
히든 레이어 수: 12
히든 사이즈: 768
어텐션 헤드 수: 12
최대 시퀀스 길이: 512
총 파라미터 수: 109,482,240 (109.5M)


### **예제 2: 모델 사용해보기**

In [9]:
# 예시 텍스트로 모델 테스트
text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# 모델 추론
with torch.no_grad():
    outputs = model(**inputs)

# 결과 분석
last_hidden_states = outputs.last_hidden_state
print(f"입력 텍스트: {text}")
print(f"토큰화된 입력: {tokenizer.tokenize(text)}")
print(f"출력 텐서 크기: {last_hidden_states.shape}")
print(f"각 토큰의 임베딩 차원: {last_hidden_states.shape[-1]}")

입력 텍스트: The quick brown fox jumps over the lazy dog.
토큰화된 입력: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
출력 텐서 크기: torch.Size([1, 12, 768])
각 토큰의 임베딩 차원: 768


## **Masked Language Modeling(MLM)소개**
마스크된 언어 모델링 기법

- **마스크된 언어 모델링(MLM)이란?**
    - **정의** : 문장에서 일부 단어를 숨기고 예측하는 학습 방법 → 빈칸 채우기와 유사한 개념
    - **목적** : 양방향 문맥 이해 능력 향상 → 앞뒤 문맥을 모두 고려한 언어 이해
    - **방식** : [MASK] 토큰으로 단어를 가리고 원래 단어 예측 → **자기지도학습(Self-supervised Learning)**

- **언제 사용하는가?**
    - **사전 훈련 단계** : 언어 모델의 기초 학습 → BERT 모델이 대량의 텍스트에서 언어 패턴 학습
    - **모델 성능 테스트** : 학습된 모델의 언어 이해도 평가 → 문맥상 적절한 단어 예측 능력 확인
    - **실제 응용 서비스** : 자동 완성, 맞춤법 검사 등 → 검색어 추천, 문서 작성 보조
    - **교육/연구 목적** : 언어 모델 작동 원리 이해 → AI의 언어 처리 능력 시연
- **등장 배경**
    - 기존 언어 모델들의 한계점 극복이 목적
    - 전통적인 언어 모델은 단방향(왼쪽→오른쪽)으로만 단어를 예측
    - 실제 인간의 언어 이해는 앞뒤 문맥을 모두 고려
    - 예시 : BERT는 문장에서 무작위로 15%의 단어를 마스킹하여 양방향 문맥을 학습

- **마스킹 방식**
|마스킹 방식|비율|처리 방법|
|---|---|---|
|[MASK] 토큰 교체|80%|실제 [MASK] 토큰으로 대체 |
|랜덤 단어 교체|10%|다른 무작위 단어로|
|교체원본 유지|10%|원래 단어 그대로 유지|

- **학습 과정**
    1. **입력 문장 준비**: "The cat is sitting on the mat"
    2. **마스킹 적용**: "The [MASK] is sitting on the mat"
    3. **문맥 분석**: 앞뒤 단어들("The", "is", "sitting", "on", "the", "mat") 고려
    4. **단어 예측**: 가능한 단어들의 확률 분포 계산
    5. **정답과 비교**: 원래 단어("cat")와 예측 결과 비교하여 학습

### **예제 1: 마스크된 언어 모델링 체험**

In [10]:
from transformers import pipeline

# 마스크 채우기 파이프라인 생성
fill_mask = pipeline("fill-mask", model="google-bert/bert-base-uncased")

# 예시 문장들
sentences = [
    "The capital of France is [MASK].",
    "I love to eat [MASK] for breakfast.",
    "The [MASK] is shining brightly today."
]

print("=== 마스크된 언어 모델링 체험 ===")
for sentence in sentences:
    result = fill_mask(sentence)
    print(f"\n입력: {sentence}")
    print("예측된 단어들:")
    for i, prediction in enumerate(result[:3], 1):
        print(f"  {i}. {prediction['token_str']} (확률: {prediction['score']:.3f})")

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


=== 마스크된 언어 모델링 체험 ===

입력: The capital of France is [MASK].
예측된 단어들:
  1. paris (확률: 0.417)
  2. lille (확률: 0.071)
  3. lyon (확률: 0.063)

입력: I love to eat [MASK] for breakfast.
예측된 단어들:
  1. it (확률: 0.135)
  2. them (확률: 0.086)
  3. you (확률: 0.072)

입력: The [MASK] is shining brightly today.
예측된 단어들:
  1. sun (확률: 0.740)
  2. moon (확률: 0.029)
  3. city (확률: 0.023)


### **예제 2: 문맥의 중요성 이해**

In [20]:
# 동일한 단어가 다른 문맥에서 어떻게 예측되는지 비교
context_examples = [
    "The [MASK] is flying in the sky.",      # 새, 비행기 등
    "The [MASK] is swimming in the ocean.",  # 물고기, 고래 등
    "The [MASK] is running in the park.",    # 사람, 개 등
]

print("\n=== 문맥에 따른 예측 변화 ===")
for sentence in context_examples:
    print(f"\n문장: {sentence}")
    results = fill_mask(sentence)
    top_3 = [f"'{r['token_str']}'" for r in results[:3]]
    print(f"상위 3개 예측: {', '.join(top_3)}")


=== 문맥에 따른 예측 변화 ===

문장: The [MASK] is flying in the sky.
상위 3개 예측: 'sun', 'moon', 'plane'

문장: The [MASK] is swimming in the ocean.
상위 3개 예측: 'girl', 'boy', 'man'

문장: The [MASK] is running in the park.
상위 3개 예측: 'train', 'horse', 'track'


### **예제 3: 실시간 MLM 체험 도구**

In [21]:
def interactive_mlm():
    """사용자가 직접 문장을 입력하여 MLM을 체험할 수 있는 함수"""
    fill_mask = pipeline("fill-mask", model="google-bert/bert-base-uncased")

    print("=== 대화형 마스크된 언어 모델링 ===")
    print("문장에 [MASK]를 포함하여 입력하세요. 종료하려면 'quit'를 입력하세요.")

    while True:
        user_input = input("\n문장 입력: ")

        if user_input.lower() == 'quit':
            print("프로그램을 종료합니다.")
            break

        if '[MASK]' not in user_input:
            print("문장에 [MASK]를 포함해주세요!")
            continue

        try:
            results = fill_mask(user_input)
            print(f"\n입력: {user_input}")
            print("예측 결과:")

            for i, result in enumerate(results[:5], 1):
                word = result['token_str']
                score = result['score']
                completed_sentence = user_input.replace('[MASK]', word)
                print(f"{i}. {completed_sentence} (신뢰도: {score:.3f})")

        except Exception as e:
            print(f"오류가 발생했습니다: {e}")

# 함수 실행 (주석 해제하여 사용)
interactive_mlm()

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


=== 대화형 마스크된 언어 모델링 ===
문장에 [MASK]를 포함하여 입력하세요. 종료하려면 'quit'를 입력하세요.

문장 입력: I'm wearing [MASK].

입력: I'm wearing [MASK].
예측 결과:
1. I'm wearing jeans. (신뢰도: 0.135)
2. I'm wearing it. (신뢰도: 0.127)
3. I'm wearing this. (신뢰도: 0.078)
4. I'm wearing nothing. (신뢰도: 0.057)
5. I'm wearing heels. (신뢰도: 0.045)

문장 입력: I'm wearing a [MASK].

입력: I'm wearing a [MASK].
예측 결과:
1. I'm wearing a dress. (신뢰도: 0.361)
2. I'm wearing a bra. (신뢰도: 0.090)
3. I'm wearing a robe. (신뢰도: 0.083)
4. I'm wearing a suit. (신뢰도: 0.064)
5. I'm wearing a shirt. (신뢰도: 0.029)

문장 입력: quit
프로그램을 종료합니다.




---

